In [3]:
using LinearAlgebra

In [4]:
function norm_2_usual(x)
  sqSum = 0
  
  for element in x
    sqSum += element^2
  end  
  
  return sqrt(sqSum)
end

norm_2_usual (generic function with 1 method)

In [5]:
# Q1.1: Norma de um vetor em Rn
function norm_2(v)
  beta = maximum(abs.(v))  
  
  if beta == 0
    return 0
  end  
  
  w = 1/beta * v
  return beta * norm_2_usual(w)  
end

norm_2 (generic function with 1 method)

In [6]:
# TEST norm_2
# v onde a norm_2_usual falha
# v = [] ?

In [7]:
# Q1.2: Rotações de Givens
function givens_rotation(f, g)
    r = norm_2([f, g])
    return f/r, g/r, r
end

givens_rotation (generic function with 1 method)

In [8]:
# TEST Rotações de Givens
v = rand(2)
c, s, r = givens_rotation(v[1], v[2])
r = round(r; digits=9)

rot = [c s; -s c]
result = round.((rot * v); digits=9)
display(result == [r; 0])

true

In [9]:
# Q1.3: Refletores de Householder
function householder_reflector(w)
  if w == 0
    return 0, 0
  end
  tau = norm_2(w)
  r = zeros(length(w))
  r[1] = tau
  v = w - r
  gamma = 2/(norm_2(v)^2)
  return gamma, v, tau
end

householder_reflector (generic function with 1 method)

In [10]:
# TEST Reflectores de HouseHolder
n = 5
w = rand(n)
gamma, v, tau = householder_reflector(w)

Q = Array{Float64}(I, (n, n)) - gamma * (v * transpose(v))
result = round.(Q * w; digits=9)
tao = round(tau; digits=9)

display(result == [tao; zeros(n-1)])

true

In [11]:
# # Q1.4 Decomposição QR = A(nxm)
# function qr_decomposition(A)
#   n, m = size(A)
#   Q = Array{Float64}(I, (n, n))
#   for k = 1:m
    
#     # Guardamos u em A para economizar memória
#     gamma, A[k:n, k], tau = householder_reflector(A[k:n, k])
#     A[k,k] = - tau
    
#     v = gamma * (transpose(u) * A[k:n, k+1:n])
#     A[k:n, k+1:n] -= (u * v)
    
#     IDN = Array{Float64}(I, (k, k))
#     N = zeros(k, n-k)
#     Q *= [IDN N; N' ]
#   end
#   R = Q * A
#   return Q, R
# end

In [12]:
# TEST QR


In [13]:
# Q2.2 Redução a forma de Hessemberg
function upper_hessenberg_form(A)
  n = size(A, 1)
  Q = Array{Float64}(I, (n, n))
  for k = 1:n-2
    beta = maximum(abs.(A[k+1:n, k]))
    if beta != 0
      ### Setting up Qk and Q
      A[k+1:n, k] /= beta
      tao = norm(A[k+1:n, k], 2) # norm() is the euclidean norm
      if A[k+1, k] < 0 
        tao = - tao
      end
      eta = A[k+1, k] + tao
      A[k+1, k] = 1
      A[k+2:n, k] /= eta
      gamma = eta/tao
      tao *= beta
      IDN = Array{Float64}(I, (k, k))
      N = zeros(k, n-k)
      v = A[k+1:n, k]
      Qk = Array{Float64}(I, (n-k, n-k)) - gamma * (v * transpose(v))
      Q *= [IDN N; N' Qk]
      ###
       
      ### Multiply on the left by Qk
      b = transpose(A[k+1:n, k]) * A[k+1:n, k+1:n]
      b *= - gamma
      A[k+1:n, k+1:n] += (A[k+1:n, k] * b)  
      ###

      ### Multiply on the right by Qk
      b = A[:, k+1:n] * A[k+1:n, k]
      b *= -gamma 
      A[:, k+1:n] += (b * transpose(A[k+1:n, k]))
      ###   

      A[k+1,k] = - tao
      A[k+2:n,k] = zeros(n-k-1)
    end
  end
  return A, Q
end

upper_hessenberg_form (generic function with 1 method)

In [14]:
# # test H e Q corretas
# A = rand(4,4)
# display(A)
# B = hessenberg(A)
# H, Q = upper_hessenberg_form(A)
# display(B)
# display(H)
# display(Q'Q)
# display(Q*H*Q')

In [157]:
# 2.3 Iteração de Francis de grau 2
# 2.3.1 Autovalores de uma matriz 2x2
function eigva22(A)
  s = sum((abs(element) for element in A))
  A *= 1/s

  t = tr(A)/2
  C = A - t * Array{Float64}(I, (2,2))
  d = ((A[1,1] - t) * (A[2,2] - t)) - (A[1,2] * A[2,1])

  eigva = [t + sqrt(abs(d)), t - sqrt(abs(d))]
  if d > 0
    eigva = [t + (1im * sqrt(d)), t - (1im * sqrt(d))]
  end

  return eigva * s
end

eigva22 (generic function with 1 method)

In [146]:
# TEST eigva22
A = rand(2,2)
eigva_calc = eigva22(A)
eigva = eigen(A).values

eigva_calc = abs.(round.(eigva_calc, digits=9))
eigva = abs.(round.(eigva, digits=9))

display(eigva_calc)
display(eigva)

2-element Array{Float64,1}:
 0.862272701
 0.294925241

2-element Array{Float64,1}:
 0.294925241
 0.862272701

In [16]:
# 2.3.3 Shift de Rayleigh generalizado
function get_shifts(H)
  n = size(H, 1)
  H_canto = H[n-1:n, n-1:n]
  shifts = eigva22(H_canto)
  if shifts isa Array{Float64,1}
    dif = abs.(shifts - [H[n,n], H[n,n]])
    s_w = shifts[argmin(dif)]
    shifts = [s_w, s_w]
  end
  return shifts
end


get_shifts (generic function with 1 method)

In [158]:
# 2.3.4 Aplicando Qk tal que (Q∗)(p(H)e1) = βe1
function get_p(H)
  shifts = get_shifts(H)
  
  ### calculando p
  first_line = ((H[1,1] - shifts[1]) * (H[1,1] - shifts[2])) + (H[1,2]*H[2,1])
  second_line = H[2,1] * ((H[1,1] + H[2,2]) - (shifts[1] + shifts[2]))
  third_line = H[3,2] * H[2,1]
  remaining = zeros(n-3)
  p = [first_line; second_line; third_line; remaining]
  ###
  
  return p
end

get_p (generic function with 1 method)

In [219]:
# TEST get_p
n = 5
A = rand(n,n)
H, Q = upper_hessenberg_form(A)
p = get_p(H)
shifts = get_shifts(H)

ID = Array{Float64}(I, (n, n))
p_calc = (H - shifts[1]*ID)*(H - shifts[2]*ID)[1:n,1]

display(round.(p, digits=9) == round.(p_calc, digits=9))


true

In [97]:
# # 2.3.5 Voltando a forma de Hessenberg
# function get_back_hessenberg(H)
#   Q = Array{Float64}(I, (n, n))
#   for k = 1:n-2
#     ID = Array{Float64}(I, (k, k))
#     N = zeros(k, n-k)
#     gamma, u, tau = householder_reflector(p)
#     Qk = Array{Float64}(I, (n-k, n-k)) - gamma * (u * transpose(u))
#     Q *= [ID N; N' Qk]
#     H = Q*H*Q
#     display(H)
#   end
#   return G, Q
# end


get_back_hessenberg (generic function with 1 method)

In [290]:
# Francis2
function francis2(H)
  n = size(H,1)
  # p(H)e1
  p = get_p(H)
  
  # refrector Q=Q0 de p(H)e1
  gamma, u, tau = householder_reflector(p)  
  Q = Array{Float64}(I, (n, n)) - gamma * (u * transpose(u))
  H = Q'*H*Q
  ###
  
  for k = 1:n-2
    # reflectores das colunas de H
    gamma, u, tau = householder_reflector(H[k+1:n, k])
    Qk = Array{Float64}(I, (n-k, n-k)) - gamma * (u * transpose(u))
    ###
    
    ID = Array{Float64}(I, (k, k))
    N = zeros(k, n-k)
    q = [ID N; N' Qk]
    
    H = q'*H*q
    Q *= Q
  end
  return H, Q
end

francis2 (generic function with 2 methods)

In [291]:
# TEST
n = 5
A = rand(n,n)
H, Q = upper_hessenberg_form(A)
display(H)
G = copy(H)
H, Q = francis2(H)
display(Q'*G*Q)

5×5 Array{Float64,2}:
  0.0486768  -0.950838   0.606689  -0.0978674  -0.292934 
 -1.33673     2.2406    -0.438484  -0.288224   -0.0454484
  0.0        -0.674426   0.702062  -0.235547    0.289139 
  0.0         0.0       -0.292107  -0.201069    0.385149 
  0.0         0.0        0.0       -0.0641348  -0.24181  

5×5 Array{Complex{Float64},2}:
    0.0486768+0.0im    -0.950838+0.0im  …  -0.0978674+0.0im   -0.292934+0.0im
     -1.33673+0.0im       2.2406+0.0im      -0.288224+0.0im  -0.0454484+0.0im
 -6.14989e-16+0.0im    -0.674426+0.0im      -0.235547+0.0im    0.289139+0.0im
  3.45253e-17+0.0im  3.81298e-17+0.0im      -0.201069+0.0im    0.385149+0.0im
          0.0+0.0im          0.0+0.0im     -0.0641348+0.0im    -0.24181+0.0im

In [292]:
A = rand(5,5)
A = Symmetric(A)

5×5 Symmetric{Float64,Array{Float64,2}}:
 0.822523  0.500739  0.450339  0.746932  0.328531
 0.500739  0.546897  0.328062  0.156501  0.586736
 0.450339  0.328062  0.677679  0.140133  0.133751
 0.746932  0.156501  0.140133  0.378267  0.206976
 0.328531  0.586736  0.133751  0.206976  0.963069